In [59]:
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D

In [22]:
churn = pd.read_csv('../data/churn.csv')

In [3]:
churn.head(10)

,Customer ID,Gender,Senior Citizen,Partner,Dependents,tenure,Phone Service,Multiple Lines,Internet Service,Online Security,...,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn
0,7590-VHVEA,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
2,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
3,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
4,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
5,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
6,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
7,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
8,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
9,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes


In [6]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7044 entries, 0 to 7043
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Customer ID        7044 non-null   object 
 1   Gender             7044 non-null   object 
 2   Senior Citizen     7044 non-null   int64  
 3   Partner            7044 non-null   object 
 4   Dependents         7044 non-null   object 
 5   tenure             7044 non-null   int64  
 6   Phone Service      7044 non-null   object 
 7   Multiple Lines     7044 non-null   object 
 8   Internet Service   7044 non-null   object 
 9   Online Security    7044 non-null   object 
 10  Online Backup      7044 non-null   object 
 11  Device Protection  7044 non-null   object 
 12  Tech Support       7044 non-null   object 
 13  Streaming TV       7044 non-null   object 
 14  Streaming Movies   7044 non-null   object 
 15  Contract           7044 non-null   object 
 16  Paperless Billing  7044 

In [7]:
churn.describe()

,Senior Citizen,tenure,Monthly Charges
count,7044.000000,7044.000000,7044.000000
mean,0.162124,32.366695,64.756736
std,0.368590,24.560582,30.090786
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [23]:
label = churn['Churn']
churn = churn.drop(columns = ['Customer ID', 'Churn'])

# numerical_columns_selector = selector(dtype_exclude = object)
# categorical_columns_selector = selector(dtype_include = object)
# numerical_columns = numerical_columns_selector(churn)
# categorical_columns = categorical_columns_selector(churn)
# print(numerical_columns)
# print(categorical_columns)

categorical = ['Gender', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 
               'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 
               'Tech Support', 'Streaming TV', 'Streaming Movies', 'Contract', 
               'Paperless Billing', 'Payment Method', 'Total Charges' , 'Senior Citizen']
numerical = ['tenure', 'Monthly Charges']

In [24]:
categorical_preprocessor = OneHotEncoder(handle_unknown = 'ignore')
numerical_preprocessor = StandardScaler()

In [27]:
ct = ColumnTransformer(
    [
        ('one-hot-encoder', categorical_preprocessor, categorical),
        ('standard_scaler', numerical_preprocessor, numerical)
    ]
)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(churn, label, random_state = 42, test_size = 0.2)

In [35]:
model = make_pipeline(ct, LogisticRegression(max_iter = 500, class_weight = 'balanced'))

In [36]:
cv_score = cross_val_score(model, X_train, y_train, cv = 5)

In [40]:
cv_score

array([0.76929902, 0.78438332, 0.74977817, 0.74977817, 0.76131322])

In [38]:
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Gender', 'Partner',
                                                   'Dependents',
                                                   'Phone Service',
                                                   'Multiple Lines',
                                                   'Internet Service',
                                                   'Online Security',
                                                   'Online Backup',
                                                   'Device Protection',
                                                   'Tech Support',
                                                   'Streaming TV',
                                                   'Streaming Movies',
                                                   'Contract',
                                                   'Paperless Billing',
                                                   'Payment Method',
                                                   'Total Charges',
                                                   'Senior Citizen']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['tenure',
                                                   'Monthly Charges'])])),
                ('logisticregression',
                 LogisticRegression(class_weight='balanced', max_iter=500))])

In [39]:
model.score(X_test, y_test)

0.765791341376863

In [44]:
churn = pd.read_csv('../data/churn.csv')

In [45]:
X = pd.get_dummies(churn.drop(['Churn', 'Customer ID'], axis = 1))
y = churn['Churn']

In [46]:
X.head()

,Senior Citizen,tenure,Monthly Charges,Gender_Female,Gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,Phone Service_No,...,Total Charges_995.35,Total Charges_996.45,Total Charges_996.85,Total Charges_996.95,Total Charges_997.65,Total Charges_997.75,Total Charges_998.1,Total Charges_999.45,Total Charges_999.8,Total Charges_999.9
0,0,1,29.85,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,29.85,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,34,56.95,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,53.85,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,45,42.30,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [89]:
churn = pd.read_csv('../data/churn.csv')
label = churn['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
churn = churn.drop(columns = ['Customer ID', 'Churn'])

categorical = ['Gender', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 
               'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 
               'Tech Support', 'Streaming TV', 'Streaming Movies', 'Contract', 
               'Paperless Billing', 'Payment Method', 'Total Charges' , 'Senior Citizen']
numerical = ['tenure', 'Monthly Charges']

categorical_preprocessor = OneHotEncoder(handle_unknown = 'ignore')
numerical_preprocessor = StandardScaler()

ct = ColumnTransformer(
    [
        ('one-hot-encoder', categorical_preprocessor, categorical),
        ('standard_scaler', numerical_preprocessor, numerical)
    ]
)

new_churn = pd.DataFrame(ct.fit_transform(churn).toarray())

In [90]:
new_churn.head()

,0,1,2,3,4,5,6,7,8,9,...,6566,6567,6568,6569,6570,6571,6572,6573,6574,6575
0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.277206,-1.160130
1,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.277206,-1.160130
2,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.066506,-0.259458
3,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.236487,-0.362487
4,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.514410,-0.746352


In [91]:
X_train, X_test, y_train, y_test = train_test_split(new_churn, label, random_state = 42, test_size = 0.2)

In [51]:
model = LogisticRegression(max_iter = 500, class_weight = 'balanced')
model.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500)

In [52]:
model.score(X_test, y_test)

0.765791341376863

In [54]:
X_train.shape

(5635, 6576)

In [92]:
model = Sequential()
model.add(Input(shape = len(X_train.columns)))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [93]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)

model.fit(X_train, y_train, batch_size = 64, validation_split = 0.1, epochs = 200)

Epoch 1/200


/opt/homebrew/lib/python3.10/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


80/80 [==============================] - 1s 7ms/step - loss: 0.5118 - accuracy: 0.7482 - val_loss: 0.4927 - val_accuracy: 0.7606
Epoch 2/200
80/80 [==============================] - 0s 2ms/step - loss: 0.4026 - accuracy: 0.8125 - val_loss: 0.4911 - val_accuracy: 0.7642
Epoch 3/200
80/80 [==============================] - 0s 2ms/step - loss: 0.3746 - accuracy: 0.8276 - val_loss: 0.4976 - val_accuracy: 0.7589
Epoch 4/200
80/80 [==============================] - 0s 2ms/step - loss: 0.3129 - accuracy: 0.8722 - val_loss: 0.5051 - val_accuracy: 0.7695
Epoch 5/200
80/80 [==============================] - 0s 2ms/step - loss: 0.2086 - accuracy: 0.9406 - val_loss: 0.4964 - val_accuracy: 0.7518
Epoch 6/200
80/80 [==============================] - 0s 3ms/step - loss: 0.0990 - accuracy: 0.9779 - val_loss: 0.6335 - val_accuracy: 0.6773
Epoch 7/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0568 - accuracy: 0.9817 - val_loss: 0.5728 - val_accuracy: 0.7606
Epoch 8/200
80/80 [======

Epoch 59/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0126 - accuracy: 0.9939 - val_loss: 1.3383 - val_accuracy: 0.6667
Epoch 60/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0121 - accuracy: 0.9947 - val_loss: 1.2224 - val_accuracy: 0.6720
Epoch 61/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0124 - accuracy: 0.9947 - val_loss: 1.2033 - val_accuracy: 0.7145
Epoch 62/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0159 - accuracy: 0.9923 - val_loss: 1.4171 - val_accuracy: 0.6241
Epoch 63/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0131 - accuracy: 0.9931 - val_loss: 1.2736 - val_accuracy: 0.6649
Epoch 64/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0134 - accuracy: 0.9939 - val_loss: 1.4330 - val_accuracy: 0.6525
Epoch 65/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0117 - accuracy: 0.9953 - val_loss: 1.2031 - val_accuracy: 0.7128
Epoch 

80/80 [==============================] - 0s 2ms/step - loss: 0.0056 - accuracy: 0.9972 - val_loss: 1.4988 - val_accuracy: 0.6968
Epoch 117/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0078 - accuracy: 0.9974 - val_loss: 1.5972 - val_accuracy: 0.7039
Epoch 118/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0065 - accuracy: 0.9966 - val_loss: 1.6191 - val_accuracy: 0.6702
Epoch 119/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0057 - accuracy: 0.9968 - val_loss: 1.8510 - val_accuracy: 0.6454
Epoch 120/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0054 - accuracy: 0.9978 - val_loss: 1.5512 - val_accuracy: 0.6933
Epoch 121/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0057 - accuracy: 0.9970 - val_loss: 1.4880 - val_accuracy: 0.7004
Epoch 122/200
80/80 [==============================] - 0s 2ms/step - loss: 0.0053 - accuracy: 0.9970 - val_loss: 1.7283 - val_accuracy: 0.6596
Epoch 123/200

80/80 [==============================] - 0s 5ms/step - loss: 0.0048 - accuracy: 0.9974 - val_loss: 1.8046 - val_accuracy: 0.6720
Epoch 174/200
80/80 [==============================] - 0s 4ms/step - loss: 0.0041 - accuracy: 0.9972 - val_loss: 1.6797 - val_accuracy: 0.6844
Epoch 175/200
80/80 [==============================] - 0s 5ms/step - loss: 0.0045 - accuracy: 0.9974 - val_loss: 2.3207 - val_accuracy: 0.6028
Epoch 176/200
80/80 [==============================] - 0s 5ms/step - loss: 0.0051 - accuracy: 0.9978 - val_loss: 1.9275 - val_accuracy: 0.6489
Epoch 177/200
80/80 [==============================] - 0s 5ms/step - loss: 0.0044 - accuracy: 0.9976 - val_loss: 1.9281 - val_accuracy: 0.6454
Epoch 178/200
80/80 [==============================] - 0s 5ms/step - loss: 0.0043 - accuracy: 0.9978 - val_loss: 1.7712 - val_accuracy: 0.6915
Epoch 179/200
80/80 [==============================] - 0s 5ms/step - loss: 0.0040 - accuracy: 0.9972 - val_loss: 1.7743 - val_accuracy: 0.6986
Epoch 180/200

In [94]:
model.evaluate(X_test, y_test)

45/45 [==============================] - 0s 849us/step - loss: 1.8100 - accuracy: 0.6757


[1.809989094734192, 0.6756564974784851]